In [2]:
import $ivy.`com.storm-enroute:scalameter_2.12:0.19`

Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter_2.12/0.19/scalameter_2.12-0.19.pom
Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter_2.12/0.19/scalameter_2.12-0.19.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/jline/jline/3.10.0/jline-3.10.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/jline/jline/3.10.0/jline-3.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/mongodb/scala/mongo-scala-driver_2.12/2.2.0/mongo-scala-driver_2.12-2.2.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/io/spray/spray-json_2.12/1.3.2/spray-json_2.12-1.3.2.pom
Downloaded https://repo1.maven.org/maven2/io/spray/spray-json_2.12/1.3.2/spray-json_2.12-1.3.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/mongodb/scala/mongo-scala-driver_2.12/2.2.0/mongo-scala-driver_2.12-2.2.0.pom
Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter-core_2.12/0.19/scalameter-core_2.12-0.19.pom.sha1
Downloaded https://repo1.maven.org/ma

Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.9.1.2/jackson-parent-2.9.1.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-modules-base/2.9.9/jackson-modules-base-2.9.9.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-base/2.9.9/jackson-base-2.9.9.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-modules-base/2.9.9/jackson-modules-base-2.9.9.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.9.1.2/jackson-parent-2.9.1.2.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-base/2.9.9/jackson-base-2.9.9.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/oss-parent/34/oss-parent-34.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-bom/2.9.9/jackson-bom-2.9.9.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-bom/2.9.9/jackson-

Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter-core_2.12/0.19/scalameter-core_2.12-0.19.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-core/2.9.9/jackson-core-2.9.9.jar
Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter-core_2.12/0.19/scalameter-core_2.12-0.19.jar
Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter-core_2.12/0.19/scalameter-core_2.12-0.19-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.9.9/jackson-databind-2.9.9.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/storm-enroute/scalameter-core_2.12/0.19/scalameter-core_2.12-0.19-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-parser-combinators_2.12/1.0.4/scala-parser-combinators_2.12-1.0.4.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-parser-combinators_2.12/1.0.4/scala-parser-combinators_2.12-1.0

Downloaded https://repo1.maven.org/maven2/org/mongodb/scala/mongo-scala-bson_2.12/2.2.0/mongo-scala-bson_2.12-2.2.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/mongodb/scala/mongo-scala-bson_2.12/2.2.0/mongo-scala-bson_2.12-2.2.0-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-core/2.9.9/jackson-core-2.9.9-sources.jar
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.9.9/jackson-module-paranamer-2.9.9-sources.jar
Downloaded https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-async/3.6.0/mongodb-driver-async-3.6.0-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-async/3.6.0/mongodb-driver-async-3.6.0-sources.jar
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.9.9/jackson-module-paranamer-2.9.9-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-ann

import $ivy.$                                       

In [3]:

import java.util.concurrent._
import scala.util.DynamicVariable


val forkJoinPool = new ForkJoinPool

abstract class TaskScheduler {
def schedule[T](body: => T): ForkJoinTask[T]
def parallel[A, B](taskA: => A, taskB: => B): (A, B) = {
  val right = task {
    taskB
  }
  val left = taskA
  (left, right.join())
}
}

class DefaultTaskScheduler extends TaskScheduler {
def schedule[T](body: => T): ForkJoinTask[T] = {
  val t = new RecursiveTask[T] {
    def compute = body
  }
  Thread.currentThread match {
    case wt: ForkJoinWorkerThread =>
      t.fork()
    case _ =>
      forkJoinPool.execute(t)
  }
  t
}
}

val scheduler =
new DynamicVariable[TaskScheduler](new DefaultTaskScheduler)

def task[T](body: => T): ForkJoinTask[T] = {
scheduler.value.schedule(body)
}

def parallel[A, B](taskA: => A, taskB: => B): (A, B) = {
scheduler.value.parallel(taskA, taskB)
}

def parallel[A, B, C, D](taskA: => A, taskB: => B, taskC: => C, taskD: => D): (A, B, C, D) = {
val ta = task { taskA }
val tb = task { taskB }
val tc = task { taskC }
val td = taskD
(ta.join(), tb.join(), tc.join(), td)
}



import java.util.concurrent._

import scala.util.DynamicVariable



forkJoinPool: ForkJoinPool = java.util.concurrent.ForkJoinPool@d63f0dc[Running, parallelism = 4, size = 0, active = 0, running = 0, steals = 0, tasks = 0, submissions = 0]
defined class TaskScheduler
defined class DefaultTaskScheduler
scheduler: DynamicVariable[TaskScheduler] = DynamicVariable(ammonite.$sess.cmd2$Helper$DefaultTaskScheduler@5aab1ead)
defined function task
defined function parallel
defined function parallel

In [5]:
val length = 10000000
val input = (0 until length).map(_ % 100 * 1.0f).toArray
val output = new Array[Float](length + 1)

length: Int = 10000000
input: Array[Float] = Array(
  0.0F,
  1.0F,
  2.0F,
  3.0F,
  4.0F,
  5.0F,
  6.0F,
  7.0F,
  8.0F,
  9.0F,
  10.0F,
  11.0F,
  12.0F,
  13.0F,
  14.0F,
  15.0F,
  16.0F,
  17.0F,
  18.0F,
  19.0F,
  20.0F,
  21.0F,
  22.0F,
  23.0F,
  24.0F,
  25.0F,
  26.0F,
  27.0F,
  28.0F,
  29.0F,
  30.0F,
  31.0F,
  32.0F,
  33.0F,
  34.0F,
  35.0F,
  36.0F,
  37.0F,
...
output: Array[Float] = Array(
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
  0.0F,
...

In [24]:

import org.scalameter._

object LineOfSightRunner {
  
  val standardConfig = config(
    Key.exec.minWarmupRuns -> 40,
    Key.exec.maxWarmupRuns -> 80,
    Key.exec.benchRuns -> 100,
    Key.verbose -> true
  ) withWarmer(new Warmer.Default)

  def main(args: Array[String]) {
    val length = 10000000
    val input = (0 until length).map(_ % 100 * 1.0f).toArray
    val output = new Array[Float](length + 1)
    //val seqtime = standardConfig measure {
    //  LineOfSight.lineOfSight(input, output)
    //}
    //println(s"sequential time: $seqtime ms")

    //val partime = standardConfig measure {
    //  LineOfSight.parLineOfSight(input, output, 10000)
    //}
    //println(s"parallel time: $partime ms")
    //println(s"speedup: ${seqtime / partime}")
  }
}

object LineOfSight {

  def max(a: Float, b: Float): Float = if (a > b) a else b

  def lineOfSight(input: Array[Float], output: Array[Float]): Unit = {
      var i = 1
      output(0) = 0
      while (i < input.length) {
          output(i) = math.atan((input(i)-input(0))/i)
          i = i + 1
      }
      println(output)
      output.scanLeft(Float.MinValue)((a,b)=>if (a>b) a else b)
  //    //angles.scanLeft(Float.MinValue)((a,b) => if (a > b) a else b)
  }
//
//  sealed abstract class Tree {
//    def maxPrevious: Float
//  }
//
//  case class Node(left: Tree, right: Tree) extends Tree {
//    val maxPrevious = max(left.maxPrevious, right.maxPrevious)
//  }
//
//  case class Leaf(from: Int, until: Int, maxPrevious: Float) extends Tree
//
//  /** Traverses the specified part of the array and returns the maximum angle.
//   */
//  def upsweepSequential(input: Array[Float], from: Int, until: Int): Float = {
//    ???
//  }
//
//  /** Traverses the part of the array starting at `from` and until `end`, and
//   *  returns the reduction tree for that part of the array.
//   *
//   *  The reduction tree is a `Leaf` if the length of the specified part of the
//   *  array is smaller or equal to `threshold`, and a `Node` otherwise.
//   *  If the specified part of the array is longer than `threshold`, then the
//   *  work is divided and done recursively in parallel.
//   */
//  def upsweep(input: Array[Float], from: Int, end: Int,
//    threshold: Int): Tree = {
//    ???
//  }
//
//  /** Traverses the part of the `input` array starting at `from` and until
//   *  `until`, and computes the maximum angle for each entry of the output array,
//   *  given the `startingAngle`.
//   */
//  def downsweepSequential(input: Array[Float], output: Array[Float],
//    startingAngle: Float, from: Int, until: Int): Unit = {
//    ???
//  }
//
//  /** Pushes the maximum angle in the prefix of the array to each leaf of the
//   *  reduction `tree` in parallel, and then calls `downsweepSequential` to write
//   *  the `output` angles.
//   */
//  def downsweep(input: Array[Float], output: Array[Float], startingAngle: Float,
//    tree: Tree): Unit = {
//    ???
//  }
//
//  /** Compute the line-of-sight in parallel. */
//  def parLineOfSight(input: Array[Float], output: Array[Float],
//    threshold: Int): Unit = {
//    ???
//  }
}



cmd24.sc:37: type mismatch;
 found   : Double
 required: Float
          output(i) = math.atan((input(i)-input(0))/i)
                               ^Compilation Failed

: 

In [19]:
LineOfSight.lineOfSight(input, output)

[F@76553489


res23: Double = 45.0